# Signal denoising

Author: Julian Lißner<br>
For questions and feedback write a mail to: [lissner@mib.uni-stuttgart.de](mailto:lissner@mib.uni-stuttgart.de)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import sys
sys.path.extend( ['provided_functions', 'incomplete_functions', '../submodules'] )
import filters
import result_check as check
from general_functions import tic, toc

## Pre processing
- on denoising, the boundary of size $\frac{\text{window_length}}2$ has to be omitted
- padding of the signal can reduce unwanted behaviour
- denoising is made up of three steps:<br>
padding, smoothing, un-padding

---
__Task:__ Implement the 'periodic_padding' and 'un_padding' functions in the 'padding.py' file.

In [ ]:
check.periodic_padding()
check.un_padding()

## Moving average
- the moving average filters a signal by computing the average (or mean) over neighbouring points in a window
- it is defined as
$$ \widehat s_i = \frac1{2m+1}\sum\limits_{j=i-m}^{i+m} \bar s_{j} $$
with $\,\widehat s_i$: denoised signal; $\qquad m$: $\frac{\rm window length}2$ <br>´
- __Note__ that $m$ is called $N$ in the lecture
- boundary values can not be computed because: for $i=1$ and $m>1 \,\blacktriangleright\,i-m <0$ 
- padding allows computation on the whole signal length<br>
- after smoothing the signal has to be unpadded


----------
__Task:__ Implement the 'moving_average' in 'filters.py'.

In [ ]:
check.moving_average( filters.moving_average)

- averaging on a window of size $2m+1$ can be also seen as a convolution with a kernel of size $2m+1$<br>
$\quad \blacktriangleright$ signal denoising just a convolution with a smoothing kernel
- the implementation with convolutions via Fourier transform is significantly faster
- if periodicity is assumed, padding is not required 
- if you have implemented periodic padding in _moving_average_, you can check if the results match

-----
__Task:__ Implement the 'moving_average_conv' in 'filters.py'.

In [ ]:
signal = np.random.rand( 128)
check.moving_average(filters.moving_average_conv )
tic( 'naive implemetation', silent=True )
a = filters.moving_average( signal, window_length=5)
toc( 'naive implemetation', precision=5 )
tic( 'convolution implemetation', silent=True )
b = filters.moving_average_conv( signal, window_length=5)
toc( 'convolution implemetation', precision=5 )
## if you deployed periodic padding uncomment the following line
#assert np.allclose( a, b ), 'moving_average_conv and moving_average do not yield the same result'

#### Moving Average performance
- the filter quality is highly affected by the window length<br>
$\quad$ smaller window length $\blacktriangleright$ less noise damped<br>
$\quad$ bigger window length $\blacktriangleright$ more information lost
- finding a good approximation to the original signal requires finetuning of the parameters
- for `window_length=1` and `window_length=signal_length` interesting phenomena can be observed

----------
__Task:__ Change the parameters and test out the moving average.

In [ ]:
# Parameters to adjust
noise_magnitude = 0.05
n_steps         = 50
window_length   = 1 

x                = np.arange( 0, 2*np.pi, 2*np.pi/n_steps )
original_signal  = np.sin( x) - np.cos(x) 
noise            = np.random.randn( n_steps) * noise_magnitude 
signal           = noise + original_signal 
#smoothed_signal = filters.#TODO( signal, window_length)

fig, axes = plt.subplots( 1, 2, figsize=(10,5))
axes[0].plot( x, signal, label='noisy signal')
axes[0].plot( x, original_signal, alpha=0.5, label='original signal')
axes[1].plot( x, smoothed_signal, label='recovered signal')
axes[1].plot( x, original_signal, alpha=0.5, label='original signal')

for ax in axes:
    ax.legend()
    ax.grid()
    ax.set_xlabel( 'x [-]' )
    ax.set_ylabel( 'y=f(x) [-]' )

-------------
---------------
## Savitzky-Golay filtering
- same principle: use neighbourhood information to smooth signal
- moving average: $\blacktriangleright$ linear 'kernel'
- Savitzky-Golay: $\blacktriangleright$ polynomial 'kernel'
- a smoothing matrix $\underline{\underline{ \beta}}$ will be defined, which is of shape  `(poly order+1, window length)`
- the smoothing matrix can compute derivatives up to order of `poly order`
- in each row of $\underline{\underline{ \beta}}$, a filtering kernel for the requested derivative is set. <br> 
$\quad \blacktriangleright$ filter array can be precomputed
- the coefficients for the filtering are computed in multiple steps
    - discretize the interval with $\xi_k = k,\quad k\in\{-m,...,m\}$, $\quad$with $m =\frac{\rm window length}2$
    - for each polynomial order compute the coefficients via $p(\xi)_{\rm order} = \xi^{\rm order}$, allocate into 2d-array $\underline{\underline P}$ (in each row one polynomial order)
    - compute the smoothing/derivative coefficients $\underline{\underline{ \beta}} = \big(\underline{\underline P}\, \underline{\underline P}^T \big)^{-1}\underline{\underline P}$

-----
__Task:__ Implement the function `compute_filter` in 'filters.py'.

In [ ]:
window_length = 7
poly_order    = 2
check.precompute_savgol( poly_order, window_length) 

#### application of the filter
- can be done with for loop, convolution or matrix multiplication<br>
- requirements for matrix multiplication (via sparse matrices) already implemented
- the computation of the $j$-th derivative of the noisy signal $\bar s$ is given as $\dfrac{\partial^j \bar s}{\partial\xi^j} \,= \beta_j * \bar s\,\dfrac {j!}{h^j} \qquad$ <br> with stepsize $h$ on the discrete signal
---
__Task:__ Implement the `apply_filter` function in 'filters.py'. Do a visual inspection of the results.<br>
The derivatives will most likely have many _spikes_.


In [ ]:
noise_magnitude = 0.05
n_steps         = 50

stepsize        = 2*np.pi/n_steps 
x               = np.arange( 0, 2*np.pi, stepsize)
original_signal = (np.sin( x) - np.cos(x)  )
signal          =  original_signal + np.random.randn( n_steps) * 0.05
derivative_1    = np.cos(x) + np.sin(x)
derivative_2    = -np.sin(x) + np.cos( x) 

precomputed_filter = filters.compute_filter( poly_order=3, window_length=7 )
smoothed_signal    = filters.apply_filter( signal, precomputed_filter)
derivatives        = filters.apply_filter( signal, precomputed_filter, derivative=[1,2], stepsize=stepsize  )

fig, axes = plt.subplots( 1,3, figsize=(15,5))
axes[0].plot( x, smoothed_signal, color='black', lw=1.5, label='smoothed signal' )
axes[0].plot( x, signal, color='blue', alpha=0.5, label='noisy signal' )
axes[0].plot( x, original_signal, color='blue', ls='--', alpha=0.5, label='original signal' )
axes[1].plot( x, derivative_1, color='red', lw=1.5, label='true first derivative' )
axes[1].plot( x, derivatives[:,0], color='black', lw=1.0, label='SavGol first derivative' )
axes[2].plot( x, derivative_2, color='red', lw=1.5, label='true second derivative' )
axes[2].plot( x, derivatives[:,1], color='black', lw=1.0, label='SavGol second derivative' )

titles = [ 'smoothed signal', 'noiseless first derivative', 'noiseless second derivative'] 
for ax in axes:
    ax.grid()
    ax.legend()
    ax.set_xlabel( 'x [-]' )
    ax.set_ylabel( 'y=f(x) [-]' )
    ax.set_title( titles.pop( 0) )

#### Performance of the Savitzky-Golay
- high polynomial orders leads to oscillations for small window lengths (c.f. interpolation)
- depending on window length and poly order, the Savitzky Golay filter can be worse than the moving average
- to compare results, we define the Mean Squared Error (MSE):
$$ MSE = \frac1n \sum\limits_{i=0}^{n-1} (x - \hat x)^2 $$
with $x$: solution  and   $\hat x$: approximation

------------
__Task__: Define the MSE. Apply the Savitzky-Golay filter and compare it to the mean filter. Adjust and try out different parameters.

In [ ]:
n_steps         = 128
mse             = lambda x, x_hat: #TODO
#TODO also adjust these paramters
noise_magnitude = 0.05 
poly_order      = [1, 2, 4]
window_length   = [ 5, 5, 55]

x               = np.arange( 0, 2*np.pi, 2*np.pi/n_steps )
original_signal = np.sin( x) - np.cos(x) 
noise           = np.random.randn( n_steps) * noise_magnitude 
signal          = noise + original_signal 

fig, axes     = plt.subplots( 1, len( poly_order), figsize=(15,5) )
print( 'noisy signal <-> original signal MSE:{:.6f}'.format( mse( original_signal, signal) ))
for i in range( len( window_length) ):
    mean_filtered = filters.moving_average( signal, window_length[i] )
    mse_averaging = mse( mean_filtered, original_signal)
    print()
    print( 'moving average window={}, MSE: {:.6f}'.format( window_length[i], mse_averaging) )    

    print( 'Savitzy- Golay MSE:' )
    for poly in poly_order:
        precomputed_filter = #TODO
        savgol_filtered    = #TODO
        mse_savgol         = #TODO #compute the MSE w.r.t. original_signal

        print( '\t window={}, poly={}, MSE: {:.6f}'.format( window_length[i], poly, mse_savgol) ) 
        axes[i].plot( x, savgol_filtered, label='savgol poly order: {}'.format( poly ) )
    axes[i].plot( x, mean_filtered, label='moving average')

    axes[i].set_title( 'window length {}'.format( window_length[i] ))
    axes[i].legend()

## Derivatives the Savitzky-Golay filter
- higher order derivatives can be immediately computed
- noise, usually very random and steep, creates spikes in the derivatives
- generally poor higher order derivatives
- signals can be filtered multiple times, each time, generally more information is lost, but more noise is reduced<br>
- multiple instances of smoothing and derivative computation yield smooth derivatives

---------------
__Task:__ Change the parameters to obtain a good approximation of the second derivative.
A MSE $\approx 0.01$ can be achieved.

In [ ]:
noise_magnitude = 0.15
n_steps         = 50

stepsize        = 2*np.pi/n_steps 
x               = np.arange( 0, 2*np.pi, stepsize)
original_signal = (np.sin( x) - np.cos(x)  )
signal          = original_signal + np.random.randn( n_steps) * 0.05
derivative_1    = np.cos(x) + np.sin(x)
derivative_2    = -np.sin(x) + np.cos( x) 

precomputed_filter = filters.compute_filter( poly_order=3, window_length=7 )
smoothed_signal    = filters.apply_filter( signal, precomputed_filter, derivative=[0,1])
first_derivative = smoothed_signal[ :,1] 
#TODO... #add more steps of signal processing
second_derivative = #TODO
#Note that the second derivative is the first derivative of the first derivative

# Compare the second derivatives
mse_derivative2 = mse( derivative_2, second_derivative)
print('MSE for second derivative: {}'.format(mse_derivative2))

### Plot
fig, axes = plt.subplots( 1,3, figsize=(15,5))
axes[0].plot( x, smoothed_signal, color='black', lw=1.5, label='smoothed signal' )
axes[0].plot( x, signal, color='blue', alpha=0.5, label='noisy signal' )
axes[0].plot( x, original_signal, color='blue', ls='--', alpha=0.5, label='original signal' )
axes[1].plot( x, derivative_1, color='red', lw=1.5, label='true first derivative' )
axes[1].plot( x, first_derivative, color='black', lw=1.0, label='SavGol first derivative' )
axes[2].plot( x, derivative_2, color='red', lw=1.5, label='true second derivative' )
axes[2].plot( x, second_derivative, color='black', lw=1.0, label='SavGol second derivative' )

titles = [ 'smoothed signal', 'first derivative', 'second derivative'] 
for ax in axes:
    ax.grid()
    ax.legend()
    ax.set_xlabel( 'x [-]' )
    ax.set_ylabel( 'y=f(x) [-]' )
    ax.set_title( titles.pop( 0) )
    ax.set_ylim( ymin=-1.5, ymax=1.5 )


## Scipy implementation
- scipy also comes with the Savitzky-Golay filter
- the function is found under `scipy.signal.savgol_filter` <br>
$\quad$ use `help( savgol_filter)` for reference

---
__Task:__ Compare your implementation to the Savitzy-Golay implementation to scipys implementation

In [ ]:
from scipy.#TODO
#help( savgol_filter) 


scipy_filtered = #TODO

fig, ax = plt.subplots(figsize=(8,6))
ax.plot( x, smoothed_signal, color='black', lw=1.5, label='manual implementation' )
ax.plot( x, scipy_filtered, color='red', lw=1.0, label='scipy implementation' )
ax.plot( x, signal, color='blue', alpha=0.5, label='noisy signal' )
ax.plot( x, original_signal, color='blue', ls='--', alpha=0.5, label='original signal' )
ax.grid()
ax.legend()